In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import seaborn as sn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as sp
from sklearn.linear_model import LogisticRegression


In [ ]:
df = pd.read_json('../input/whats-cooking-kernels-only/train.json')
df.set_index('id',inplace=True)
df.sort_index(ascending=True,inplace=True)

In [ ]:
mlb = MultiLabelBinarizer()
binarized = mlb.fit_transform(df['ingredients'])
sparse_binarized = sp.csr_matrix(binarized)

In [ ]:
cuisine_encoder = LabelEncoder()

In [ ]:
X = sparse_binarized
Y = cuisine_encoder.fit_transform(df['cuisine'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
...     sparse_binarized, Y, test_size=0.20, random_state=42)

In [ ]:
logistic_regression = LogisticRegression(max_iter=4000)
logistic_regression.fit(X_train,y_train)

In [ ]:
test = pd.read_json("../input/whats-cooking-kernels-only/test.json")

In [ ]:
%%capture --no-stderr
to_predict = mlb.transform(test['ingredients'])

In [ ]:
sparse_to_predict = sp.csr_matrix(to_predict)
predictions = logistic_regression.predict(sparse_to_predict)
cuisine_encoder.inverse_transform(predictions)
test['cuisine'] = cuisine_encoder.inverse_transform(predictions)
test = test.drop('ingredients',axis=1)

In [ ]:
filename_output = './submission.csv'
test.to_csv(filename_output, index=False)